In [ ]:
%matplotlib inline
%pip install -q matplotlib-scalebar
import numpy as np
import matplotlib.pyplot as plt 
from pathlib import Path
from aicsimageio import AICSImage
from aicsimageio.writers import OmeTiffWriter
from mmv_im2im.preprocessing import transforms
from monai.transforms import NormalizeIntensity, CenterSpatialCrop, Compose
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import mean_squared_error
import scipy as sc
import random
from matplotlib.patches import Rectangle
from matplotlib_scalebar.scalebar import ScaleBar
import matplotlib.gridspec as gridspec
import copy
import monai
import torch

compression_techniques = ['JPEGXR','JPEG_2000_LOSSY','LERC']
holdout_path_2d = Path("/mnt/eternus/users/Jan/data/CompressAI/examples/data/labelfree2D/FBL/holdout")

In [12]:
# input= dir structure: types and raw data
def plot_data(root_dir):
    #Collect all used methods
    print(root_dir)
    #random_image= random.choice(list(root_dir.glob('*_IM.tiff')))
    random_image= next(iter(root_dir.glob('*7922_IM.tiff')))
    #random_image_backup= copy.deepcopy(random_image)
    print(random_image)
    for dir in [i for i in holdout_path_2d.glob('*') if (i / random_image.name).exists() or (i / str(random_image.stem+'_decoded.tiff')).exists()   ]:
        try:
            print(dir)  
            # Find all immediate subdirectories
            subdirectories =  [subdir for subdir in dir.iterdir() if subdir.is_dir()]

            #plt.figure(figsize=(20,9)) # specifying the overall grid size
            plt.figure(figsize=(10, 6), dpi=300)
            plt.subplots_adjust(wspace=0.1, hspace=0.25)
            #plt.suptitle(dir.name) # or plt.suptitle('Main title')  
            input_img = AICSImage(random_image).get_image_data("YX")
            

            if dir.name in compression_techniques:
                compression_img = AICSImage(dir / random_image.name).get_image_data("YX")
            else:
                compression_img = AICSImage(dir / str(random_image.stem+'_decoded.tiff')).get_image_data("YX")


            # Plot
            plt.subplot(2, 2, 1)
            plt.imshow(input_img, cmap='gray')
            plt.title(f'Input')
            scalebar1 = ScaleBar(0.108333, 'um', frameon=False, color='white')
            plt.gca().add_artist(scalebar1)
            plt.axis('off')

            plt.subplot(2, 2, 2)
            plt.imshow(compression_img, cmap='gray')
            plt.title('Compression')
            scalebar2 = ScaleBar(0.108333, 'um', frameon=False, color='white')
            plt.gca().add_artist(scalebar2)
            plt.axis('off')

            if len(subdirectories):
                if dir.name in compression_techniques:
                    prediction_img = AICSImage(dir / 'prediction' / Path(random_image.stem+'_pred.tiff')).get_image_data("YX")
                else:
                    prediction_img = AICSImage(dir / 'prediction' / Path(random_image.stem+'_decoded_pred.tiff')).get_image_data("YX")
                groundtrouth_img = AICSImage(root_dir / random_image.name.replace('IM','GT')).get_image_data("YX")
                original_prediction = AICSImage(dir.parent/ 'original' / 'prediction' / Path(random_image.stem+'_pred.tiff')).get_image_data("YX")
                
                plt.subplot(2, 2, 3)
                plt.imshow(prediction_img, cmap='gray')
                plt.title('Prediction')
                scalebar3 = ScaleBar(0.108333, 'um', frameon=False, color='white')
                plt.gca().add_artist(scalebar3)
                plt.axis('off')

                plt.subplot(2, 2, 4)
                gt = NormalizeIntensity()(groundtrouth_img)
                plt.imshow(gt, cmap='gray')
                plt.title('Ground Truth (Normalized Intensity)')
                scalebar5 = ScaleBar(0.108333, 'um', frameon=False, color='white')
                plt.gca().add_artist(scalebar5)
                plt.axis('off')
        except FileNotFoundError as e:
            print(e)
            continue
            

        # plt.savefig(dir.name + '_plot.png')

In [ ]:
plot_data(holdout_path_2d)

In [63]:
# input= dir structure: types and raw data
def plot_data(root_dir):
    #Collect all used methods
    print(root_dir)
    #random_image= random.choice(list(root_dir.glob('*_IM.tiff')))
    random_image= next(iter(root_dir.glob('*7922_IM.tiff')))
    #random_image_backup= copy.deepcopy(random_image)
    print(random_image)
    for dir in [i for i in holdout_path_2d.glob('*') if (i / random_image.name).exists() or (i / str(random_image.stem+'_decoded.tiff')).exists()   ]:
        try:
            print(dir)  
            # Find all immediate subdirectories
            subdirectories =  [subdir for subdir in dir.iterdir() if subdir.is_dir()]
            spacing = 0.05

            # Calculate the width and height of a subplot based on the space available and desired spacing
            width = (1 - 4 * spacing) / 3
            height = (1 - 3 * spacing) / 2

            

            # To center-align the subplots in the second row, 
            # we'll adjust their x-positions based on the spacing and width.
            # Calculate the total space available for the second row
            total_space = 1 - 2 * spacing
            space_occupied_by_subplots = 2 * width
            space_left = total_space - space_occupied_by_subplots

            # Calculate the new starting x-position for the fourth subplot to center-align them
            new_start_x = spacing + space_left / 2.3

            # Adjust positions for the first row
            pos1 = [new_start_x, 0.5 + spacing, width, height]
            pos2 = [pos1[0] + width + spacing, pos1[1], width, height]
            pos3 = [spacing, spacing, width, height]
            # Adjust positions for the center-aligned subplots in the second row
            pos4 = [pos3[0] + width + spacing, spacing, width, height]
            pos5 = [pos4[0] + width + spacing, spacing, width, height]
            
            input_img = AICSImage(random_image).get_image_data("YX")
            

            if dir.name in compression_techniques:
                compression_img = AICSImage(dir / random_image.name).get_image_data("YX")
            else:
                compression_img = AICSImage(dir / str(random_image.stem+'_decoded.tiff')).get_image_data("YX")


            # Plot
            plt.figure(figsize=(12, 6))
            plt.clf()
            ax1 = plt.axes(pos1)  # 第一行第一个位置
            plt.imshow(input_img, cmap='gray')
            plt.title(f'Input')
            scalebar1 = ScaleBar(0.108333, 'um', frameon=False, color='white')
            ax1.add_artist(scalebar1)
            plt.axis('off')

            ax2 = plt.axes(pos2)
            plt.imshow(compression_img, cmap='gray')
            plt.title('Compression')
            scalebar2 = ScaleBar(0.108333, 'um', frameon=False, color='white')
            ax2.add_artist(scalebar2)
            plt.axis('off')

            if len(subdirectories):
                if dir.name in compression_techniques:
                    prediction_img = AICSImage(dir / 'prediction' / Path(random_image.stem+'_pred.tiff')).get_image_data("YX")
                else:
                    prediction_img = AICSImage(dir / 'prediction' / Path(random_image.stem+'_decoded_pred.tiff')).get_image_data("YX")
                groundtrouth_img = AICSImage(root_dir / random_image.name.replace('IM','GT')).get_image_data("YX")
                original_prediction = AICSImage(dir.parent/ 'original' / 'prediction' / Path(random_image.stem+'_pred.tiff')).get_image_data("YX")
                
                ax3 = plt.axes(pos3)
                plt.imshow(prediction_img, cmap='gray')
                plt.title('Compressed Prediction')
                scalebar3 = ScaleBar(0.108333, 'um', frameon=False, color='white')
                ax3.add_artist(scalebar3)
                plt.axis('off')

                ax4 = plt.axes(pos4)
                gt = NormalizeIntensity()(original_prediction)
                plt.imshow(gt, cmap='gray')
                plt.title('Original Prediction')
                scalebar4 = ScaleBar(0.108333, 'um', frameon=False, color='white')
                ax4.add_artist(scalebar4)
                plt.axis('off')
                
                ax5 = plt.axes(pos5)
                gt = NormalizeIntensity()(groundtrouth_img)
                plt.imshow(gt, cmap='gray')
                plt.title('Ground Truth (Normalized Intensity)')
                scalebar5 = ScaleBar(0.108333, 'um', frameon=False, color='white')
                ax5.add_artist(scalebar5)
                plt.axis('off')
            plt.show()
        except FileNotFoundError as e:
            print(e)
            continue

In [ ]:
plot_data(holdout_path_2d)